In [ ]:
import os
os.chdir('/your path/multiomics-cardiovascular-disease')
print(os.getcwd())

import sys
sys.path.append('/your path/multiomics-cardiovascular-disease/data_loader')
sys.path

In [ ]:
import logging
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from sksurv.util import Surv
from sksurv.nonparametric import kaplan_meier_estimator
from lifelines import KaplanMeierFitter
from data_preparation import UKBiobankDataMerge

In [ ]:
data_dir = '/your path/multiomics-cardiovascular-disease/data/'
figures_dir = '/your path/multiomics-cardiovascular-disease/figures'
log_filename = os.path.join(figures_dir, 'FollowupInformation/FollowupInformation.log')
logging.basicConfig(level=logging.INFO, filename=log_filename, filemode='w', format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
data = UKBiobankDataMerge(data_dir=data_dir, logger=logger)

In [ ]:
ukb_df = data.get_merged_data(excluded=True)
ukb_df

# Median followup time

In [ ]:
# Calculate the range of follow-up time
min_follow_up_time = ukb_df['bl2cved_yrs'].min()
max_follow_up_time = ukb_df['bl2cved_yrs'].max()
logger.info("Minimum follow-up time: %.4f years", min_follow_up_time)
logger.info("Maximum follow-up time: %.4f years", max_follow_up_time)

# Calculate median follow-up time
median_follow_up_time = ukb_df['bl2cved_yrs'].median()
logger.info("Median follow-up time: %.4f years", median_follow_up_time)

density_plot = sns.kdeplot(ukb_df['bl2cved_yrs'], fill=False)
density_y_values = density_plot.get_lines()[0].get_ydata()
density_x_values = density_plot.get_lines()[0].get_xdata()
# median_y_value = density_y_values[density_x_values.tolist().index(min(density_x_values, key=lambda x: abs(x - median_follow_up_time)))]

# Plot the density plot
plt.figure(figsize=(10, 6))
density_plot = sns.kdeplot(ukb_df['bl2cved_yrs'], fill=True, color='#8491B4')
# plt.axvline(median_follow_up_time, color='gray', linestyle='--', ymax=median_y_value / max(density_y_values)-0.0378)
plt.axvline(median_follow_up_time, color='gray', linestyle='--', ymax=0.88)

# Add the median follow-up time
plt.text(plt.xlim()[0] + 0.5, plt.ylim()[0] + 0.025, f'Median: {median_follow_up_time:.1f} years', color='black', fontsize=12)

# Add labels
plt.xlabel('Observation time (years)', fontsize=12)
plt.ylabel('Density', fontsize=12)

# Save the figure
output_pdf_dir = os.path.join(figures_dir, 'FollowupInformation/Median_followup_time.pdf')
output_png_dir = os.path.join(figures_dir, 'FollowupInformation/Median_followup_time.png')
plt.savefig(output_pdf_dir, format='pdf', dpi=300, bbox_inches='tight')
plt.savefig(output_png_dir, format='png', dpi=300, bbox_inches='tight')

plt.show()

# CVD-free survival (Cardiovascular diseases + Cardiovascular death)

In [ ]:
male_df = ukb_df[ukb_df['male'] == 1]
female_df = ukb_df[ukb_df['male'] == 0]

kmf_male = KaplanMeierFitter()
kmf_female = KaplanMeierFitter()

kmf_male.fit(durations=male_df['bl2cved_yrs'], event_observed=male_df['cved'], label='Male')
kmf_female.fit(durations=female_df['bl2cved_yrs'], event_observed=female_df['cved'], label='Female')

In [ ]:
male_data = kmf_male.confidence_interval_survival_function_.copy()
male_data['timeline'] = kmf_male.survival_function_.index
male_data['survival_prob'] = kmf_male.survival_function_['Male']
male_data['sex'] = 'Male'

male_data = male_data.rename(columns={
    'Male_lower_0.95': 'lower_0.95',
    'Male_upper_0.95': 'upper_0.95'
})

female_data = kmf_female.confidence_interval_survival_function_.copy()
female_data['timeline'] = kmf_female.survival_function_.index
female_data['survival_prob'] = kmf_female.survival_function_['Female']
female_data['sex'] = 'Female'

female_data = female_data.rename(columns={
    'Female_lower_0.95': 'lower_0.95',
    'Female_upper_0.95': 'upper_0.95'
})

combined_data = pd.concat([male_data[['timeline', 'survival_prob', 'lower_0.95', 'upper_0.95', 'sex']],
                           female_data[['timeline', 'survival_prob', 'lower_0.95', 'upper_0.95', 'sex']]], 
                          axis=0).reset_index(drop=True)
combined_data.to_csv('/your path/MultiomicsCVD/figures/FollowupInformation/cvd_free_survival.csv', index=False)
combined_data